# The Project
- QNN for XOR Problem:

  Classiq has an available dataset for training PQC to imitate the XOR gate, similar to how we trained a U-gate to act as a NOT gate. Design a QNN to solve the XOR problem. Read more on the dataset [here](https://docs.classiq.io/latest/reference-manual/built-in-algorithms/qml/qnn/datasets/#datasetxor).

  Click [here](https://en.wikipedia.org/wiki/XOR_gate) for more information about XOR gates



## Setting The Scene

When running from Google Colab, we need to install Classiq's SDK and authenticate the remote device:

In [18]:
#!pip install classiq numpy torchvision torch


In [57]:
# import classiq
# classiq.authenticate(overwrite=True)

# Step 1 - Create our `torch.nn.Module`

## Step 1.1 - Create Dataset

We will use the DATALOADER_NOT dataset.

In [127]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
# XORゲートのデータセット
from torch.utils.data import DataLoader
from classiq.applications.qnn.datasets import DATALOADER_XOR

for data, label in DATALOADER_XOR:
    print(f"--> Data for training:\n{data}")
    print(f"--> Corresponding labels:\n{label}")



--> Data for training:
tensor([[0., 1.],
        [1., 1.],
        [0., 0.],
        [1., 0.]])
--> Corresponding labels:
tensor([1., 0., 0., 1.])


## Step 1.2 - Create our parametric quantum program

Our quantum model will be defined and synthesized as follows:

In [128]:
from classiq import *


import numpy as np


# 量子プログラムの作成
@qfunc
def encoding(input_0: CInt, input_1: CInt, q: QArray[QBit]) -> None:
    RX(theta=input_0*np.pi, target=q[0])
    RX(theta=input_1*np.pi, target=q[1])


@qfunc
def mixing(theta: CReal, q: QArray[QBit]) -> None:
    CRX(theta, q[1], q[0])

@qfunc
def main(input_0: CReal, input_1: CReal, weight_0: CReal, weight_1: CReal, res: Output[QArray[QBit]]) -> None:
    allocate(2, res)
    encoding(input_0, input_1, res)  # Loading input
    mixing(weight_0, res)   # Adjustable parameter


model = create_model(main)
quantum_program = synthesize(model)


#show(quantum_program)



## Step 1.3 - Create the Execution and Post-processing

The following example defines a function that takes in a parametric quantum program plus parameters, executes the program, and returns the result. Notes:

1. The code can be executed on a physical computer or on a simulator. In any case, implement the execution using `execute_qnn`.
2. Post-process the result of the execution to obtain a single number (`float`) and a single dimension `Tensor`.


In [183]:
from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.datasets import DATALOADER_NOT
import torch

# 実行およびポストプロセス関数の作成
def execute(
    quantum_program: SerializedQuantumProgram, arguments: MultipleArguments
) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

def post_process(result: SavedResult) -> torch.Tensor:
    """
    
    probability of measuring 
    p_one  |01> + |11> 
    p_zero |00> + |10>
    """
    counts: dict = result.value.counts
    p_one:float = counts.get("01", 0.0) / sum(counts.values()) + counts.get("11", 0.0) / sum(counts.values()) 
    p_zero:float = counts.get("00", 0.0) / sum(counts.values()) + counts.get("10", 0.0) / sum(counts.values())
    p_xor: float = p_one - p_zero
    
    return torch.tensor(0.0 if p_xor < 0 else p_xor)




## Step 1.4 - Create a network

Now we're going to define a network, just like any other PyTorch network, only that this time, we will have only 1 layer, and it will be a quantum layer.

In [205]:
import torch.nn as nn
from classiq.applications.qnn import QLayer
# ネットワークの作成

class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,  # the quantum program, the result of `synthesize()`
            execute,  # a callable that takes
            # - a quantum program
            # - parameters to that program (a tuple of dictionaries)
            # and returns a `ResultsCollection`
            post_process,  # a callable that takes
            # - a single `SavedResult`
            # and returns a `torch.Tensor`
            *args,
            **kwargs
        )
        

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # return the new parameter
        return self.qlayer(x)

model = Net()


# Step 2 - Choose a dataset, loss function, and optimizer

We will use optimizer,defined [here](https://docs.classiq.io/latest/reference-manual/built-in-algorithms/qml/qnn/datasets/) as well as [L1Loss](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html) and [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [206]:
from classiq.applications.qnn.datasets import DATALOADER_NOT
import torch.nn as nn
import torch.optim as optim
from classiq.applications.qnn.datasets import DATALOADER_XOR

data_loader=DATALOADER_XOR        

# 損失関数とオプティマイザの定義
loss_func = nn.L1Loss()     # Mean Absolute Error (MAE)
optimizer = optim.SGD(model.parameters(), lr=1)

# Step 3 - Train

For the training process, we will use a loop similar to [the one recommended by PyTorch](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#update-the-weights)

In [207]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


# 学習およびテストの準備
def train(model, data_loader, loss_func, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        print(epoch, model.qlayer.weight)
        for data, label in data_loader:
            optimizer.zero_grad()
            output = model(data)
            output = output.view(-1)  # 出力の形状を修正
            label = label.view(-1)    # ラベルの形状を修正
            loss = loss_func(output, label)
            loss.backward()

            # 勾配チェックを追加して問題を診断
            for name, param in model.named_parameters():
                if param.grad is not None:
                    print(f'Grad for {name}: {param.grad.norm().item()}')
                else:
                    print(f'No grad for {name}')
                if param.grad is not None and torch.all(param.grad == 0):
                    print(f'Gradient for {name} is zero.')

            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(data_loader)}')

    # 学習後のモデルを保存
    torch.save(model.state_dict(), './trained_model.pth')
    print("Trained model saved to trained_model.pth")
train(model, data_loader, loss_func, optimizer, epochs=100)

0 Parameter containing:
tensor([0.7832], requires_grad=True)
Grad for qlayer.weight: 0.048828125
Epoch 1/100, Loss: 0.427001953125
1 Parameter containing:
tensor([0.8320], requires_grad=True)
Grad for qlayer.weight: 0.18310546875
Epoch 2/100, Loss: 0.419677734375
2 Parameter containing:
tensor([1.0151], requires_grad=True)
Grad for qlayer.weight: 0.35400390625
Epoch 3/100, Loss: 0.376708984375
3 Parameter containing:
tensor([1.3691], requires_grad=True)
Grad for qlayer.weight: 0.732421875
Epoch 4/100, Loss: 0.299560546875
4 Parameter containing:
tensor([2.1016], requires_grad=True)
Grad for qlayer.weight: 0.30517578125
Epoch 5/100, Loss: 0.119140625
5 Parameter containing:
tensor([2.4067], requires_grad=True)
Grad for qlayer.weight: 0.52490234375
Epoch 6/100, Loss: 0.059814453125
6 Parameter containing:
tensor([2.9316], requires_grad=True)
Grad for qlayer.weight: 0.09765625
Epoch 7/100, Loss: 0.00390625
7 Parameter containing:
tensor([3.0293], requires_grad=True)
Grad for qlayer.weight

# Step 4 - Test

Lastly, we will test our network accuracy, using [the following answer](https://stackoverflow.com/questions/52176178/pytorch-model-accuracy-test#answer-64838681)

In [ ]:
# テスト関数
def check_accuracy(model: nn.Module, data_loader: DataLoader, atol=1e-4) -> float:
    num_correct = 0
    total = 0
    model.eval()

    with torch.no_grad():                        # temporarily disable gradient calculation
        for data, labels in data_loader:
            # let the model predict
            predictions = model(data)
            print('predictions:', predictions)
            print('labels:     ', labels)

            # get a tensor of booleans, indicating if each label is close to the real label
            is_prediction_correct = predictions.isclose(labels, atol=atol)

            # count the amount of `True` predictions
            num_correct += is_prediction_correct.sum().item()
            # count the total evaluations, the first dimension of `labels` is `batch_size`
            total += labels.size(0)

    print(f"num_correct: {num_correct}, total: {total}")
    accuracy = float(num_correct) / float(total)
    return accuracy


accuracy = check_accuracy(model, data_loader)

print(f"Test Accuracy of the model: {accuracy*100:.2f}%")

# デバッグ情報を追加して再度確認
for data, label in data_loader:
    output = model(data)
    label = label.view(-1)  # ラベルの形状を修正
    loss = loss_func(output, label)
    print(f"Data: {data}, Label: {label}, Output: {output}, Loss: {loss.item()}")
    break  # 最初のバッチのみを確認


## Reload model

In [203]:
# 新しいモデルインスタンスの作成
loaded_model = Net()

# モデルの読み込み
loaded_model.load_state_dict(torch.load("trained_model.pth"))
print("Model loaded from trained_model.pth")

# モデルのテスト
accuracy = check_accuracy(model, data_loader)
print(f"Test Accuracy of the model: {accuracy*100:.2f}%")

Model loaded from trained_model.pth
predictions: tensor([0.0000, 0.0000, 1.0000, 0.9990], requires_grad=True)
labels:      tensor([0., 0., 1., 1.])
num_correct: 3, total: 4
Test Accuracy of the model: 75.00%
